In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data.dataset import Dataset
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [2]:
data = np.loadtxt('train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


In [3]:
blacks = 0
whites = 0

In [4]:
for elem in data:
    if elem[0] == 'b':
        blacks += 1
    elif elem[0] == 'w':
        whites += 1

In [5]:
blacks, whites

(1019906, 921217)

In [6]:
dataBlack = np.empty(blacks, dtype = '<U718')
dataWhite = np.empty(whites, dtype = '<U718')

In [7]:
dataBlack

array(['', '', '', ..., '', '', ''], dtype='<U718')

In [8]:
curBlack = 0
curWhite = 0

In [9]:
data.dtype

dtype('<U718')

In [10]:
for i in range(data.shape[0]):
    if data[i][0] == 'b':
        dataBlack[curBlack] = data[i]
        curBlack += 1
    elif data[i][0] == 'w':
        dataWhite[curWhite] = data[i]
        curWhite += 1

In [11]:
np.savetxt('dataBlack.txt', dataBlack, delimiter='\n', fmt="%s")

In [12]:
np.savetxt('dataWhite.txt', dataWhite, delimiter='\n', fmt="%s")

In [13]:
data = None
dataWhole = np.concatenate((dataBlack, dataWhite))
print(dataWhole.shape)

(1941123,)


In [14]:
np.savetxt('dataWhole.txt', dataWhole, delimiter='\n', fmt="%s")


In [2]:
data = np.loadtxt('dataWhole.txt', dtype=str, delimiter = '\n')


In [3]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [4]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

def toBoards(gameBad, color):
    game = gameBad.split()
    res = []
    labels = []
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, len(game)):
        curBoard = np.zeros(shape=(3, 15, 15))
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        turn = toTurn(game[i])
        if i % 2 == 1 and color == "black":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
            labels.append(turn[0] * 15 + turn[1])
        elif i % 2 == 0 and color == "white":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
            labels.append(turn[0] * 15 + turn[1])

        
        if curPlayer == 1:
            blackBoard[turn] = 1
        else:
            whiteBoard[turn] = -1


        curPlayer *= -1
        #printBoard(curBoard)
    return np.asarray(res), np.asarray(labels)

In [5]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

#### Black

In [9]:
fileSizeBlack = 1019906 // 103
print(fileSizeBlack)
fileSizeWhite = 921217 // 83
print(fileSizeWhite)

9902
11099


In [11]:
curFile = 0
for curFile in range(103):
    for gm in range(10):
        ansFeatures = np.ndarray(shape=(0, 3, 15, 15), dtype=np.int8)
        ansLabels = np.ndarray(shape=0, dtype=np.int16)
        for i in range(1000):
            res, labels = toBoards(dataBlack[curFile * fileSizeBlack + gm * 1000 + i], "black")
            ansFeatures = np.concatenate((ansFeatures, res), axis=0)
            ansLabels = np.concatenate((ansLabels, labels), axis=0)

            
        print(ansFeatures.shape)
        curStrFeatures = "poses/blackF" + str(curFile) + ".npy"
        curStrLabels = "poses/blackL" + str(curFile) + ".npy"

        np.save(curStrFeatures, ansFeatures)
        np.save(curStrLabels, ansLabels)

        curFile += 1

    

NameError: name 'dataBlack' is not defined

In [12]:
class CustomDatasetFromGames(Dataset):
    def __init__(self, numBlack, numWhite=0):
        # Read data
        self.feature_arr = np.concatenate([np.load("poses/blackF" + str(i) + ".npy") for i in range(numBlack)])
        self.label_arr = np.concatenate([np.load("poses/blackL" + str(i) + ".npy") for i in range(numBlack)])
        
        # Calculate len
        self.data_len = (self.feature_arr).shape[0]
        
    def __getitem__(self, index):
        # Get ndarray from index
        feature = self.feature_arr[index]
        label = self.label_arr[index]        
        return (torch.from_numpy(feature).float(), label)

    def __len__(self):
        return self.data_len


In [14]:
datasetBlack = CustomDatasetFromGames(30)
trainLoaderBlack = torch.utils.data.DataLoader(dataset=datasetBlack, batch_size=32, shuffle=False)

In [58]:
def takeRandom():
    game = data[np.random.randint(data.shape[0])]
    print(game)

In [60]:
takeRandom()

black h8 j9 j6 g7 g10 j7 k7 k8 h6 h5 h10 h9 j10 f10 l8 k9 m9 n10 l10 k10 g9 j11 k11 j12 h12 l9 n7 n8 m6 l6 l5 k4 m5 m7 k5 n11 m10 n12 n9 n13 n14 o9 l11 m12 m11 l12


In [ ]:
def makeBatch(size):
    batch = np.zeros(shape=(size, 3, 15, 15))
    labels = np.zeros(shape=size)
    for i in range(size):
        batch[i,:,:,:], labels[i] = takeRandom()
    return from_numpy(batch), from_numpy(labels)

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(3, affine=False)
        self.pad2 = nn.ConstantPad2d(2, 0)
        self.pad1 = nn.ConstantPad2d(1, 0)

        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 4, 5)
        self.conv3 = nn.Conv2d(4, 2, 5)
        self.conv4 = nn.Conv2d(2, 1, 3)

    def forward(self, x):
        x = self.batchnorm(x.float())
        x = self.pad1(F.relu(self.conv1(x)))
        x = self.pad2(F.relu(self.conv2(x)))
        x = self.pad2(F.relu(self.conv3(x)))
        x = self.pad1(F.relu(self.conv4(x)))

        x = x.view(-1, 225)
 
        return x

In [30]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
batchNum = 100000
epochNum = 10
batchSize = 32
optimizer = optim.Adam(net.parameters(), lr=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [31]:
for epoch in range(epochNum):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(batchNum):
        # get the inputs
        inputs, labels = makeBatch(batchSize)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 4.785
[1,   400] loss: 4.315
[1,   600] loss: 4.131
[1,   800] loss: 4.113
[1,  1000] loss: 3.960
[1,  1200] loss: 3.979
[1,  1400] loss: 3.878
[1,  1600] loss: 3.802
[1,  1800] loss: 3.729
[1,  2000] loss: 3.759
[1,  2200] loss: 3.790
[1,  2400] loss: 3.655
[1,  2600] loss: 3.381
[1,  2800] loss: 2.987
[1,  3000] loss: 3.041
[1,  3200] loss: 3.306
[1,  3400] loss: 3.378
[1,  3600] loss: 3.203
[1,  3800] loss: 3.389
[1,  4000] loss: 3.247
[1,  4200] loss: 3.202
[1,  4400] loss: 3.156
[1,  4600] loss: 3.450
[1,  4800] loss: 3.374
[1,  5000] loss: 3.207
[1,  5200] loss: 3.337
[1,  5400] loss: 3.388
[1,  5600] loss: 3.361
[1,  5800] loss: 3.411
[1,  6000] loss: 3.268
[1,  6200] loss: 3.364
[1,  6400] loss: 3.223
[1,  6600] loss: 3.128
[1,  6800] loss: 2.632
[1,  7000] loss: 3.309
[1,  7200] loss: 3.193
[1,  7400] loss: 3.268
[1,  7600] loss: 3.289
[1,  7800] loss: 3.149
[1,  8000] loss: 3.274
[1,  8200] loss: 3.219
[1,  8400] loss: 2.957
[1,  8600] loss: 3.070
[1,  8800] 

[5,  7600] loss: 3.045
[5,  7800] loss: 3.007
[5,  8000] loss: 3.014
[5,  8200] loss: 2.952
[5,  8400] loss: 2.710
[5,  8600] loss: 2.855
[5,  8800] loss: 2.941
[5,  9000] loss: 3.085
[5,  9200] loss: 3.032
[5,  9400] loss: 3.052
[5,  9600] loss: 2.930
[5,  9800] loss: 2.860
[5, 10000] loss: 2.820
[5, 10200] loss: 3.054
[5, 10400] loss: 3.031
[5, 10600] loss: 3.092
[5, 10800] loss: 3.033
[5, 11000] loss: 3.095
[5, 11200] loss: 3.140
[5, 11400] loss: 3.085
[5, 11600] loss: 2.923
[5, 11800] loss: 2.757
[5, 12000] loss: 2.813
[5, 12200] loss: 2.563
[5, 12400] loss: 2.743
[5, 12600] loss: 2.721
[5, 12800] loss: 2.742
[5, 13000] loss: 3.003
[5, 13200] loss: 2.829
[5, 13400] loss: 2.882
[5, 13600] loss: 2.974
[5, 13800] loss: 2.865
[5, 14000] loss: 2.899
[5, 14200] loss: 2.862
[5, 14400] loss: 2.717
[5, 14600] loss: 2.726
[5, 14800] loss: 2.907
[5, 15000] loss: 2.944
[5, 15200] loss: 2.954
[5, 15400] loss: 2.818
[5, 15600] loss: 2.803
[5, 15800] loss: 2.901
[5, 16000] loss: 2.719
[6,   200] 

KeyboardInterrupt: 

## Game

In [126]:
def toTripleBoard(board, side):
    curBoard = np.zeros(shape=(3, 15, 15), dtype=np.float)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.float)
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.float)

    if side == 1:
        blackTurnBoard = np.ones(shape=(15, 15), dtype=np.float)
    else:
        blackTurnBoard = -np.ones(shape=(15, 15), dtype=np.float)
    curBoard[0,:] = blackTurnBoard
    for i in range(15):
        for j in range(15):
            if board[i, j] == 1:
                blackBoard[i, j] = 1
            if board[i, j] == -1:
                whiteBoard[i, j] = -1
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    return curBoard


In [176]:
def printBoard(curBoard):
    print("-------------------------------------------------------------")
    cprint("    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 ", "blue")
    for i in range(15):
        if i < 10:
            cprint("", i, "blue", end=" ")
        for j in range(15):
            if curBoard[i, j] == -1:
                print("|", end="")
                cprint(str(-1), 'red', "on_green", attrs=['blink'], end=" ")
            else:
                print("|", end = " ")
                if curBoard[i, j] == 1:
                    cprint(int(curBoard[i, j]), 'yellow', "on_blue", attrs=['blink'], end=" ")
                else:
                    print(int(curBoard[i, j]), end=" ")
        print("|")
        print("-------------------------------------------------------------")
    print()
    return


 ## (we are white)

In [ ]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)

    curBoard[netTurn // 15, netTurn % 15] = 1
    printBoard(curBoard)
    
    x, y = list(map(int, input().split()))
    curBoard[x, y] = -1
    #printBoard(curBoard)


-------------------------------------------------------------
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
------------------------------------------------------

10 13
-------------------------------------------------------------
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
------------------------------------------------

## (we are black)

In [ ]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    x, y = list(map(int, input().split()))
    curBoard[x, y] = 1
    #printBoard(curBoard)
    
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)

    curBoard[netTurn // 15, netTurn % 15] = -1
    #printBoard(curBoard)


In [152]:

text = colored('Hello, World!', 'red', attrs=['reverse', 'blink'])
print(text)
cprint('Hello, World!', 'green', 'on_yellow', attrs=['blink'])

Hello, World!
Hello, World!
